<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/WhaleTracker_Pro2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yfinance --quiet

import pandas as pd
import numpy as np
import yfinance as yf
import requests, zipfile, io
from datetime import datetime

# =====================================
# 0️⃣ BAIXAR AUTOMATICAMENTE COT
# =====================================

current_year = datetime.now().year
url = f"https://www.cftc.gov/files/dea/history/fut_disagg_txt_{current_year}.zip"

print("🔽 Baixando dados COT...")

resp = requests.get(url)
if resp.status_code != 200:
    raise Exception("Erro ao baixar COT da CFTC")

with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    fname = [n for n in z.namelist() if n.endswith(".txt")][0]
    data_bytes = z.read(fname)

df = pd.read_csv(io.BytesIO(data_bytes), sep=",")

print("✅ COT carregado com sucesso")

# =====================================
# 1️⃣ PREPARAÇÃO DOS DADOS
# =====================================

df["Date"] = pd.to_datetime(df["Report_Date_as_YYYY-MM-DD"])

cols_numeric = [
    "Change_in_M_Money_Long_All", "Change_in_M_Money_Short_All",
    "M_Money_Positions_Long_All", "M_Money_Positions_Short_All",
    "Open_Interest_All"
]

for col in cols_numeric:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df["M_Money_Net_Change"] = (
    df["Change_in_M_Money_Long_All"] -
    df["Change_in_M_Money_Short_All"]
)

df["M_Money_Net_%OI"] = (
    df["M_Money_Net_Change"] /
    df["Open_Interest_All"]
)

df["Flow_Z"] = df.groupby("Market_and_Exchange_Names")[
    "M_Money_Net_%OI"
].transform(lambda x: (x - x.mean()) / x.std())

df["M_Money_Net_Position"] = (
    df["M_Money_Positions_Long_All"] -
    df["M_Money_Positions_Short_All"]
)

df["M_Money_Net_Position_%OI"] = (
    df["M_Money_Net_Position"] /
    df["Open_Interest_All"]
)

df["Institutional_Score"] = (
    0.7 * df["M_Money_Net_%OI"] -
    0.3 * df["M_Money_Net_Position_%OI"]
)

# =====================================
# 2️⃣ FILTRAR ATIVOS RELEVANTES
# =====================================

relevant_assets = {
    "GOLD": "GC=F",
    "SILVER": "SI=F",
    "CRUDE OIL": "CL=F",
    "E-MINI S&P 500": "ES=F",
    "NASDAQ-100": "NQ=F",
    "EURO FX": "6E=F",
    "JAPANESE YEN": "6J=F"
}

latest_date = df["Date"].max()
latest = df[df["Date"] == latest_date]

signals = latest[
    latest["Market_and_Exchange_Names"].str.contains(
        "|".join(relevant_assets.keys()), case=False, na=False
    )
].copy()

print("\n📅 Última data COT:", latest_date)

# =====================================
# 3️⃣ BUSCAR PREÇOS
# =====================================

def get_price_data(ticker):
    return yf.download(ticker, period="3mo", interval="1d", progress=False)

results = []

for asset_name, ticker in relevant_assets.items():
    match = signals[
        signals["Market_and_Exchange_Names"].str.contains(asset_name, case=False)
    ]

    if not match.empty:
        score = match["Institutional_Score"].values[0]
        zscore = match["Flow_Z"].values[0]

        price_data = get_price_data(ticker)

        if len(price_data) > 10:
            price_data["Return_5d"] = price_data["Close"].pct_change(5)
            latest_return = price_data["Return_5d"].iloc[-1]

            results.append({
                "Asset": asset_name,
                "Score": round(score,6),
                "Flow_Z": round(zscore,2),
                "5d_Return": round(latest_return,4)
            })

results_df = pd.DataFrame(results)

# =====================================
# 4️⃣ CLASSIFICAÇÃO FINAL
# =====================================

results_df = results_df.sort_values("Score", ascending=False)

print("\n🚀 RANKING FINAL INSTITUCIONAL")
print(results_df)

print("\n🔥 SINAIS LONG (Score>0 & Z>1)")
print(results_df[(results_df["Score"] > 0) & (results_df["Flow_Z"] > 1)])

print("\n🔻 SINAIS SHORT (Score<0 & Z<-1)")
print(results_df[(results_df["Score"] < 0) & (results_df["Flow_Z"] < -1)])

🔽 Baixando dados COT...
✅ COT carregado com sucesso

📅 Última data COT: 2026-02-17 00:00:00


/tmp/ipython-input-771/3245475363.py:104: FutureWarning: YF.download() has changed argument auto_adjust default to True
  return yf.download(ticker, period="3mo", interval="1d", progress=False)
/tmp/ipython-input-771/3245475363.py:104: FutureWarning: YF.download() has changed argument auto_adjust default to True
  return yf.download(ticker, period="3mo", interval="1d", progress=False)
/tmp/ipython-input-771/3245475363.py:104: FutureWarning: YF.download() has changed argument auto_adjust default to True
  return yf.download(ticker, period="3mo", interval="1d", progress=False)



🚀 RANKING FINAL INSTITUCIONAL
       Asset     Score  Flow_Z  5d_Return
2  CRUDE OIL  0.007549   -0.22    -0.0310
1     SILVER -0.006168    1.26     0.1292
0       GOLD -0.064013    0.66     0.0423

🔥 SINAIS LONG (Score>0 & Z>1)
Empty DataFrame
Columns: [Asset, Score, Flow_Z, 5d_Return]
Index: []

🔻 SINAIS SHORT (Score<0 & Z<-1)
Empty DataFrame
Columns: [Asset, Score, Flow_Z, 5d_Return]
Index: []
